In [7]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
import os

In [8]:
# Load fine-tuned model
model_path = "t5-sql-finetuned"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [13]:
# Get the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
data_path = os.path.join(project_root, 'text_to_sql_generator', 'data', 'processed', 'text2sql_clean.csv')
df= pd.read_csv(data_path)
prompts = df["sql_prompt"].tolist()[:10000]
targets = df["sql"].tolist()[:10000]

In [14]:
# Evaluation loop
exact_matches = 0
bleu_scores = []
errors = []

for prompt, target in tqdm(zip(prompts, targets), total=len(prompts)):
    input_text = "translate to SQL: " + prompt
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to("cuda")
    outputs = model.generate(**inputs, max_length=100, num_beams=4)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Exact match
    if prediction.strip().lower() == target.strip().lower():
        exact_matches += 1
    else:
        errors.append((prompt, prediction, target))

    # BLEU
    reference = target.strip().split()
    candidate = prediction.strip().split()
    bleu = sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method1)
    bleu_scores.append(bleu)

# Results
total = len(prompts)
print(f"\n🔎 Evaluation Results:")
print(f"✅ Exact Match Accuracy: {exact_matches / total:.2%}")
print(f"🧪 Average BLEU Score: {sum(bleu_scores)/total:.4f}")
print(f"❌ Failed Predictions: {len(errors)} / {total}")

# Show sample errors
print("\nSome incorrect predictions:")
for i, (prompt, pred, true_sql) in enumerate(errors[:5]):
    print(f"\n[{i+1}] Prompt: {prompt}")
    print(f"  🔴 Predicted: {pred}")
    print(f"  ✅ Target:    {true_sql}")

100%|██████████| 10000/10000 [1:09:07<00:00,  2.41it/s]


🔎 Evaluation Results:
✅ Exact Match Accuracy: 2.43%
🧪 Average BLEU Score: 0.1617
❌ Failed Predictions: 9757 / 10000

Some incorrect predictions:

[1] Prompt: What is the total volume of timber sold by each salesperson, sorted by salesperson?
  🔴 Predicted: SELECT salesperson, SUM(volume) as total_volume FROM salespersons GROUP BY salesperson;
  ✅ Target:    SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;

[2] Prompt: List all the unique equipment types and their corresponding total maintenance frequency from the equipment_maintenance table.
  🔴 Predicted: SELECT equipment_type, SUM(maintenance_frequency) as total_maintenance FROM equipment_maintenance;
  ✅ Target:    SELECT equipment_type, SUM(maintenance_frequency) AS total_maintenance_frequency FROM equipment_maintenance GROUP BY equipment_type;

[3] Prompt: How many marine